In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Начальные параметры для модели
import re
from scipy import stats
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import string
import spacy

import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import time
nltk.downloader.download('vader_lexicon')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
RANDOM_SEED = 42 #фиксируем SEED

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

## Данные из соревнования

### Первоначальная версия датасета содержит 17 полей со следующей информацией:

- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве.
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.
- sample
- reviewer_score - в конце таблицы он меняет значение, которое становится не равным нулю

In [ ]:
INPUT_DIR = '../input/input/'
train_data = pd.read_csv(INPUT_DIR + 'hotels_train.csv') 
test_data = pd.read_csv(INPUT_DIR + 'hotels_test.csv') 
sample_submission = pd.read_csv(INPUT_DIR + 'submission.csv')


### Объединим train и test в один датасет

In [ ]:
# для корректной обработки признаков объединяем трейн и тест в один датасет
train_data['sample'] = 1 # помечаем train
test_data['sample'] = 0 # помечаем test
test_data['reviewer_score'] = 0 
# в test у нас нет значения reviewer_score, мы его должны предсказать, по этому заполняем нулями

data = test_data.append(train_data, sort=False).reset_index(drop=True)

### Анализируем данные

In [ ]:
print(data.shape)
data.tail(3)

In [ ]:
data.info()

- в признаках lat и lng имеются пропуски. т.е. не все геоданные отелей указаны.
- числовые данные в:
 - additional_number_of_scoring
 - average_score
 - review_total_negative_word_counts
 - total_number_of_reviews
 - review_total_positive_word_counts
 - total_number_of_reviews_reviewer_has_given
 - reviewer_score
 - lat
 - Ing
- признак даты в столбце review_date
- с остальными надо разбираться внимательнее

### Воспользуемся бонусом "EDA одной строкой кода")

In [ ]:
# pip install pandas-profiling

In [ ]:
#from pandas_profiling import ProfileReport
#profile = ProfileReport(data, title="Booking Pandas Profiling Report")
#profile

Анализ profile даёт нам:
- статистика датасета:
 - 18 признаков
 - 515738 строк(значений)
 - 6536 отсутствующих значений
- типы переменных:
 - 9 цифровых
 - 9 категорий
- предупреждение о:
 - 336 дубликатов строк(значений)
 - review_total_negative_word_counts имеют 127890 (24.8%) нулевых значений
 - review_total_positive_word_counts имеют 35946 (7.0%) нулевых значений
 - список признаков с высокой корреляцией
 - перечень признаков с большим количеством элементов
 #### предварительный вывод:
1. Анализ дубликатов строк показывает, что данные дублируются два раза в 100% случаях, т.е. часть таблицы загружена дважды. (перепроверить, пометить)
2. 6536 отсутствующих значений - из .info ясно, что это в признаках lat и lng имеются пропуски именно в этом количестве. Надо извлечь адрес отеля и по адресу установить координаты. (правда, не знаю как?) 
3. Нулевые значения в количестве слов в отзывах - думаю поставить наименьшее значение в признаке, т.к. негативный или позитивный отзыв всё-таки установлен, но клиент не приложил усилие по написанию слов, что, по-моему, является минимально влияющим аспектом.
4. А вот уровень эмоциональности отзыва должна сильно влиять на рейтинг отеля. Пользуясь подсказкой нашего ментора Сергея Добдина думаю воспользоваться библиотекой для обработки текстов для классификации nltk.
5. Список с высокой корреляцией отработаю.

## Разбор признаков

### 1.Oтметим дублированные строки

In [ ]:
data['duplicat'] = data.duplicated()
#def dupl(arg):
#    if False in arg:
#        return 0
#    elif True:
#        return 1
#data['duplicated'] = data['duplicat'].apply(dupl)
#data = data.drop(['duplicat'], axis=1)
data.head()
#  в итоге признак не значительный

In [ ]:
data['duplicat'].describe()

In [ ]:
# Здесь видны дубликаты
sorted = data.sort_values(by='duplicat')
sorted.tail() 

### 2. Географические координаты
Пропущены данные в столбцаx lat и lng. Это категориальные признаки. Не имеет смысла заполнять их какими-то значениями, так как это географическое положение. Заполним пропущенные значения нулями.

In [ ]:
data['lat'] = data['lat'].fillna(0, inplace=True)
data['lng'] = data['lng'].fillna(0, inplace=True)

Есть ли среди названий отелей одинаковые, но при этом находящиеся в разных городах? Проверим.

In [ ]:
name_address = data.pivot_table(
    values = 'hotel_address',
    index = 'hotel_name',
    aggfunc = 'nunique'
)
name_address.head()

In [ ]:
dupl = name_address[name_address['hotel_address'] > 1]
dupl

Да, Hotel Regina популярен в трёх местах.
Надо идентифицировать отель Hotel Regina по городам.

In [ ]:
# Cоздадим список, в котором будет храниться название отеля, имеющее несколько адресов.
dupl_list = dupl[dupl['hotel_address'] > 1].index.to_list()
dupl_list

In [ ]:
data['hotel_name'] = data.apply(lambda x: (x['hotel_name'] + ' in ' + x['hotel_address'].split()[-1])
    if (x['hotel_name'] in dupl_list) else x['hotel_name'], axis=1)
data['hotel_name'].nunique()

получили 1494 уникальных значений отелей

### Преобразуем адрес отеля

посмотрим как выглядит запись адреса в DF

In [ ]:
data['hotel_address'].head()

In [ ]:
# извлекаем признак страны из hotel_address
data['hotel_country'] = data['hotel_address'].apply(lambda x: x.split(' ')[-1])
data['hotel_country'] = data['hotel_country'].apply(lambda x: 'United Kingdom' if x == 'Kingdom' else x)
display(data['hotel_country'].value_counts())


In [ ]:
# извлекаем признак города из hotel_address 
data['hotel_city'] = data['hotel_address'].apply(lambda x: x.split(' ')[-2])
data['hotel_city'] = data['hotel_city'].apply(lambda x: 'London' if x == 'United' else x)
display(data['hotel_city'].value_counts())

Получается у нас отели из шести городов шести стран. Закодируем страну. 

In [ ]:
data['hotel_country'] = data['hotel_country'].astype('category')
data['hotel_country_code'] = data['hotel_country'].cat.codes

Близкое расположение отеля к местным достопримечательностям наверняка улучшит отзыв об отеле. Но определить расположение достопримечательности в центре города было бы не правильно. (я только что приехал из Севастополя...) Плюс основная часть туристов посещала гостиниу в течении одной ночи, что свидетельствует скорее о командировке и достопримечательности в этом случае не принимаются в расчет.
Так же у нас есть адрес отеля и этого уже достаточно.
Поэтому географические координаты удаляем.

In [ ]:
data = data.drop(['lat', 'lng'], axis=1)
data.head()

In [ ]:
# после извлечения полезных данных из hotel_address, его можно удалить
data = data.drop(['hotel_address'], axis=1)
data.head(3)

In [ ]:
# добавляем население города
сity_population = {
    'Paris': 2139907, 'London': 9002488, 'Milan': 1374582,
    'Vienna': 1951354, 'Barcelona': 1636732, 'Amsterdam': 905234
}

data['сity_population'] = data['hotel_city'].map(сity_population)

### 3. Слова в отзывах (тэги)
проанализируем:
 - кол-во уникальных тэгов
 - популярные тэги
 - количество ночей 
 - зависимость длительности проживания от количества оценок

In [ ]:
def count_tags(tags):
    express = re.findall("'[^\'](.+?)[^\']'", tags)
    return express

print('Уникальных тэгов: ', data['tags'].apply(count_tags).explode().nunique())

In [ ]:
# напишем функцию для создания словаря с подсчетом тегов
unique_dic = {}
def make_unique_dic(x):
    x = x[3:-3].split(" ', ' ")
    for elem in x:
        if elem in unique_dic:
            unique_dic[elem] = unique_dic[elem] + 1
        else:
            unique_dic[elem] = 1
    return x

data['tags'].apply(make_unique_dic)

# получим датафрейм со всеми тегами и их частотой встречаемости
unique_tags = pd.DataFrame(unique_dic, index=['count']).transpose()
display(unique_tags)

In [ ]:
# Введём новый признак count_nights (количество ночей)
def count_nights(tags):
    express = re.findall(r'Stayed\s([0-9]+)\snight', tags)
    return express

data['count_nights'] = data['tags'].apply(count_nights)

def convert(feature):
    for elem, num in enumerate(feature):
        return np.float64(num)
    
data['count_nights'] = data['count_nights'].apply(convert).convert_dtypes().fillna(1)

In [ ]:
pivot_nights = pd.pivot_table(
    data=data,
    index='count_nights', 
    values='reviewer_score', 
    aggfunc=['count', 'mean'])
fig1, ax1 = plt.subplots(figsize=(15, 7))
plt.suptitle(
    'Количество оценок в зависимости от проведённых в отеле ночей', size=20)
sns.barplot(x=pivot_nights.index, y=pivot_nights[('count', 'reviewer_score')]);

### 4. Уровень эмоциональности
целочисленные значения, которые нам подходят:
- Общее количество отзывов total_number_of_reviews_reviewer_has_given
- Признак review_total_negative_words_count и review_total_positive_word_counts
- Признак total_negative_words_count и total_positive_words_count

Признак days_since_review представлен строкой - преобразуем его и получим оттуда число.

In [ ]:
def count_days(feature):
    count = re.findall(r'\d+', feature)
    return count
    
data['days_since_review'] = data['days_since_review'].apply(count_days)
data['days_since_review'] = data['days_since_review'].apply(convert).convert_dtypes()
# Приведём review_date к типу datetime64:
data['review_date'] = pd.to_datetime(data['review_date'])

Проверим, влияет ли сезонность на количество оценок

In [ ]:
data['review_month'] = data['review_date'].dt.month
score_month = data[data['sample']==1].pivot_table(
values='reviewer_score',
index='review_month',
aggfunc='mean')
score_month

In [ ]:
fig2, ax2 = plt.subplots(figsize=(15, 5))
plt.suptitle('Среднее значение количества оценок в зависимости от месяца', size=16)
bar_month = sns.barplot(
    x=score_month.index, 
    y=score_month['reviewer_score'])
ax2.set_xticklabels([
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

for p in bar_month.patches:
    bar_month.annotate(
        '{:.2f}'.format(p.get_height()), (p.get_x()+0.4, p.get_height()),
    ha='center', va='bottom', fontsize=14)

plt.ylim(8, 8.7);

Есть просадка показателей в летний период. Предполагаю в процентном содержании становится больше отдыхающих, по сравнению с командировочными...)

### Пробуем библиотеку для обработки текстов для классификации nltk, которая различает эмоциональную тональность отзыва

In [ ]:
# Создаём объекты SentimentIntensityAnalyzer
positive = SentimentIntensityAnalyzer()
negative = SentimentIntensityAnalyzer()

# посмотрим как изменится характеристика слова 'nothing':
example = 'nothing'
print(f'Словарь оценок слова "{example}" до преобразований:')
print(negative.polarity_scores(example), '\n')

# добавим контекст:
negative_words = {'small': -1, 'nothing': 10}
positive_words = {'nothing': -10, 'everything': 10}
negative.lexicon.update(negative_words)
positive.lexicon.update(positive_words)

print(f' - после преобразований (для негативного контекста):')
print(negative.polarity_scores(example))
print(f' - после преобразований (для позитивного контекста):')
print(positive.polarity_scores(example))

# фразы "No Negative" тонально будет нести негативную окраску:
example = 'No Negative'
print(f'\nСловарь оценок фразы "{example}":')
print(negative.polarity_scores(example))
example = 'Positive'
print(f'Словарь оценок фразы "{example}":')
print(negative.polarity_scores(example))

# меняем метки отсутствия частей отзыва:
data['negative_review_temp'] = data['negative_review'].apply(lambda x: x.replace('No Negative', 'Positive'))
data['positive_review_temp'] = data['positive_review'].apply(lambda x: x.replace('No Positive', 'Negative'))

In [ ]:
%%time

data['negative_review_sentiments'] = data['negative_review_temp'].apply(lambda x: negative.polarity_scores(x))

data['positive_review_sentiments'] = data['positive_review_temp'].apply(lambda x: positive.polarity_scores(x))

# формируем новые признаки:
data['neg_review_sentiments_neg'] = data['negative_review_sentiments'].apply(lambda x: x['neg'])
data['neg_review_sentiments_neu'] = data['negative_review_sentiments'].apply(lambda x: x['neu'])
data['neg_review_sentiments_pos'] = data['negative_review_sentiments'].apply(lambda x: x['pos'])
data['neg_review_sentiments_compound'] = data['negative_review_sentiments'].apply(lambda x: x['compound'])

data['pos_review_sentiments_neg'] = data['positive_review_sentiments'].apply(lambda x: x['neg'])
data['pos_review_sentiments_neu'] = data['positive_review_sentiments'].apply(lambda x: x['neu'])
data['pos_review_sentiments_pos'] = data['positive_review_sentiments'].apply(lambda x: x['pos'])
data['pos_review_sentiments_compound'] = data['positive_review_sentiments'].apply(lambda x: x['compound'])

In [ ]:
data.info()

In [ ]:
# убираем признаки, которые ещё не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим
# избавимся от других нечисловых признаков: category, datetime64[ns]
cat_date_columns = [cd for cd in data.columns if ((data[cd].dtypes.name == 'category') or (data[cd].dtypes == 'datetime64[ns]'))]
data.drop(cat_date_columns, axis = 1, inplace=True)

In [ ]:
# убираем признаки, которые ещё не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим

object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

In [ ]:
data.info()

В итоге у нас осталось 20 признаков, представленных числовыми типами данных: int8, int64, float64 и sample.

## 5. Анализ мултиколлинеарности
разделим признаки по типу, перечислим их:

In [ ]:
cols_num = [
    'review_total_negative_word_counts',
    'review_total_positive_word_counts',
    'days_since_review',
    'total_number_of_reviews_reviewer_has_given',
    'reviewer_score',
    'сity_population',
    'neg_review_sentiments_neg',
    'neg_review_sentiments_neu',
    'neg_review_sentiments_pos',
    'neg_review_sentiments_compound',
    'pos_review_sentiments_neg',
    'pos_review_sentiments_neu',
    'pos_review_sentiments_pos',
    'pos_review_sentiments_compound'   
]

cols_cat = [
    'additional_number_of_scoring',
    'average_score',
    'total_number_of_reviews',
    'hotel_country_code',
    'count_nights',
    'review_month',
]

In [ ]:
fig_, ax_ = plt.subplots(figsize=(15, 10))
corr = data[cols_num].corr(method='pearson')
sns.heatmap(corr, 
            annot=True, 
            linewidths=0.1, 
            fmt='.1g')
ax_.set_title('Корреляция непрерывных признаков', fontsize=18)
plt.show()

Удаляем признаки с корреляцией больше 0.8

In [ ]:
del_cols = ['neg_review_sentiments_pos']

In [ ]:
fig_, ax_ = plt.subplots(figsize=(15, 10))
corr = data[cols_cat].corr(method='pearson')
sns.heatmap(corr, 
            annot=True, 
            linewidths=0.1, 
            fmt='.1g')
ax_.set_title('Корреляция категориальных признаков', fontsize=18)
plt.show()

In [ ]:
del_cols = ['total_number_of_reviews']

Удаляем признаки, попавшие в список del_cols

In [ ]:
data.drop(del_cols, axis = 1, inplace=True)

In [ ]:
print('Осталось признаков: {}'.format(data.shape[1]))

## Оценка значимости признаков
Для оценки значимости категориальных признаков будем использовать тест хи-квадрат, для непрерывных признаков — тест ANOVA.

In [ ]:
y = data.query('sample == 1').drop(['sample'], axis=1)['reviewer_score'].values
X = data.query('sample == 1').drop(['sample', 'reviewer_score'], axis=1)

# визуализируем результат анализа значимости:
imp_num = pd.Series(f_classif(X[X.columns], y)[0], index = X.columns)
imp_num.sort_values(inplace = True)

fig5, ax5 = plt.subplots(figsize=(15, 20))
imp_num.plot(kind = 'barh', color='blue');

Удалим признаки, почти не коррелирующие с целевым показателем: days_since_review, review_month , count_nights.

In [ ]:
cols_for_del = ['days_since_review', 'duplicat']

In [ ]:
data.drop(cols_for_del, axis = 1, inplace=True)

In [ ]:
print('ВСЕГО ПРИЗНАКОВ: {}'.format(data.shape[1]))

In [ ]:
data.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)